In [1]:
import os
import re
import io
import math
import numpy as np
import pandas as pd

log_dir = "./logs/fgm"   # 필요하면 여기만 바꿔

# 파일명 패턴: Opp_bumper..._Gap..._Bub..._SPD..._FGM_..._dur_...s.csv
log_pattern = re.compile(
    r"^Opp_.*"                     # Opp_로 시작
    r"_FFG[-\d.]+"                 # _FFG<fgm_fov_angle>
    r"_SCD[-\d.]+"                 # _SCD<fgm_speed_check_fov_deg>
    r"_Gap[-\d.]+"                 # _Gap<...>
    r"_Bub[-\d.]+"                 # _Bub<...>
    r"_Clr[-\d.]+"                 # _Clr<...>
    r"_WW[-\d.]+"                  # _WW<...>
    r"_AW[-\d.]+"                  # _AW<...>
    r"_SW[-\d.]+"                  # _SW<...>
    r"_HB[-\d.]+"                  # _HB<...>
    r"_CT[-\d.]+"                  # _CT<...>
    r"_SA[-\d.]+"                  # _SA<...>
    r"_DBS[-\d.]+"                 # _DBS<...>
    r"_SPD[-\d.]+"                 # _SPD<...>
    r"_FGM_.*_dur_[-\d.]+s\.csv$"  # _FGM_<...>_dur_<t>s.csv
)

all_files = [
    os.path.join(log_dir, f)
    for f in os.listdir(log_dir)
    if f.endswith(".csv") and log_pattern.match(f)
]

print(f"발견된 로그 파일 개수: {len(all_files)}")
for f in all_files[:5]:
    print(" -", os.path.basename(f))


발견된 로그 파일 개수: 12
 - Opp_bumper_slow_1.csv_FFG190.0_SCD35.0_Gap1.5_Bub0.7_Clr0.75_WW0.4_AW4.0_SW0.13_HB3.0_CT0.35_SA0.3_DBS0.22_SPD3.0_FGM_1764850010_GOAL_dur_19.19s.csv
 - Opp_bumper_slow_1.csv_FFG180.0_SCD30.0_Gap1.3_Bub0.55_Clr0.60_WW0.5_AW5.0_SW0.10_HB2.5_CT0.30_SA0.4_DBS0.15_SPD3.5_FGM_1764849969_GOAL_dur_13.61s.csv
 - Opp_bumper_slow_1.csv_FFG180.0_SCD25.0_Gap1.2_Bub0.5_Clr0.55_WW0.6_AW6.0_SW0.08_HB2.0_CT0.25_SA0.5_DBS0.12_SPD4.0_FGM_1764849949_GOAL_dur_9.90s.csv
 - Opp_bumper_slow_0.5.csv_FFG180.0_SCD25.0_Gap1.3_Bub0.6_Clr0.55_WW0.5_AW5.5_SW0.08_HB2.0_CT0.25_SA0.75_DBS0.25_SPD4.2_FGM_1764849804_ABORT_dur_0.00s.csv
 - Opp_bumper_slow_1.csv_FFG180.0_SCD25.0_Gap1.0_Bub0.4_Clr0.45_WW0.7_AW8.0_SW0.05_HB1.5_CT0.20_SA0.6_DBS0.18_SPD5.5_FGM_1764849990_GOAL_dur_9.86s.csv


In [2]:
def load_run_log(path: str):
    """
    하나의 run_logger csv 파일을
    - df_data: per-step 데이터 (t,x,y,...)
    - summary_dict: 맨 아래 요약 블록 (key,value)
    로 나눠서 반환.
    """
    with open(path, "r") as f:
        lines = f.readlines()
    
    # Summary 시작 라인 찾기
    summary_idx = None
    for i, line in enumerate(lines):
        if line.strip().startswith('--- Experiment Summary'):
            summary_idx = i
            break
    
    if summary_idx is None:
        # Summary 없는 경우 (비상용)
        data_text = "".join(lines)
        df = pd.read_csv(io.StringIO(data_text))
        summary = {}
    else:
        # 맨 위 ~ summary 직전 빈 줄까지만 데이터로 사용
        data_text = "".join(lines[:summary_idx-1])
        df = pd.read_csv(io.StringIO(data_text))
        
        # summary 부분은 "key,value" 형식
        summary_lines = [l.strip() for l in lines[summary_idx+1:] if l.strip()]
        summary = {}
        for l in summary_lines:
            if "," in l:
                k, v = l.split(",", 1)
                summary[k.strip()] = v.strip()
    
    return df, summary


In [3]:
def parse_params_from_filename(path: str):
    """
    파일 이름에서
      scenario, Gap, Bub, Clr, WW, AW, SW, HB, CT, SA, DBS, SPD, result, duration_from_name
    를 뽑아서 dict로 반환.
    """
    name = os.path.basename(path)
    
    # 파라미터 블럭 파싱
    param_pattern = re.compile(
        r"_FFG(?P<FFG>[-\d.]+)"
        r"_SCD(?P<SCD>[-\d.]+)"
        r"_Gap(?P<Gap>[-\d.]+)"
        r"_Bub(?P<Bub>[-\d.]+)"
        r"_Clr(?P<Clr>[-\d.]+)"
        r"_WW(?P<WW>[-\d.]+)"
        r"_AW(?P<AW>[-\d.]+)"
        r"_SW(?P<SW>[-\d.]+)"
        r"_HB(?P<HB>[-\d.]+)"
        r"_CT(?P<CT>[-\d.]+)"
        r"_SA(?P<SA>[-\d.]+)"
        r"_DBS(?P<DBS>[-\d.]+)"
        r"_SPD(?P<SPD>[-\d.]+)"
    )

    
    m = param_pattern.search(name)
    params = {}
    if m:
        for k, v in m.groupdict().items():
            try:
                params[k] = float(v)
            except ValueError:
                params[k] = None
    
    # scenario (Gap 앞부분 전체)
    scen = name.split("_Gap")[0]
    params["scenario"] = scen   # ex) Opp_bumper_slow_0.5.csv
    
    # 결과 태그 (GOAL / CRASH / STUCK / ABORT 등)
    m2 = re.search(r"_(GOAL|CRASH|STUCK|ABORT)_", name)
    if m2:
        params["result"] = m2.group(1)
    
    # 파일명에 있는 duration
    m3 = re.search(r"_dur_([0-9.]+)s\.csv$", name)
    if m3:
        try:
            params["duration_from_name"] = float(m3.group(1))
        except ValueError:
            params["duration_from_name"] = None
    
    return params


In [4]:
def parse_summary_types(summary: dict):
    """
    summary dict 의 value를 적당히 숫자/불리언으로 캐스팅.
    """
    out = {}
    for k, v in summary.items():
        v = v.strip()
        # 숫자 시도
        try:
            out[k] = float(v)
            continue
        except ValueError:
            pass
        
        # YES/NO -> 1/0
        if v.upper() in ("YES", "NO"):
            out[k] = 1 if v.upper() == "YES" else 0
        else:
            out[k] = v
    return out


In [5]:
def compute_metrics(df: pd.DataFrame, safe_dist: float = 0.5):
    """
    run_logger 데이터 df 에서
    - total_time
    - collision_flag
    - min_dist / unsafe_ratio
    - fgm_min_dist / fgm_unsafe_ratio / fgm_track_rms
    를 계산 (있으면).
    """
    metrics = {}
    
    # 총 시간
    if "t" in df.columns and len(df) > 1:
        metrics["total_time"] = float(df["t"].iloc[-1] - df["t"].iloc[0])
    else:
        metrics["total_time"] = 0.0
    
    # 충돌 플래그
    if "collision" in df.columns:
        metrics["collision_flag"] = int(df["collision"].max() > 0)
    else:
        metrics["collision_flag"] = 0
    
    # 전역 min_d / unsafe_ratio
    if "min_d" in df.columns:
        metrics["min_dist"] = float(df["min_d"].min())
        metrics["unsafe_ratio"] = float((df["min_d"] < safe_dist).mean())
    
    # FGM 전용
    if "planner" in df.columns and "fgm_min_d" in df.columns:
        mask_fgm = df["planner"].astype(str).str.contains("FGM")
        if mask_fgm.any():
            df_fgm = df[mask_fgm]
            metrics["fgm_min_dist"] = float(df_fgm["fgm_min_d"].min())
            metrics["fgm_unsafe_ratio"] = float((df_fgm["fgm_min_d"] < safe_dist).mean())
            if "fgm_track" in df_fgm.columns:
                metrics["fgm_track_rms"] = float(math.sqrt((df_fgm["fgm_track"] ** 2).mean()))
    
    return metrics


In [6]:
def analyze_log(path: str, safe_dist: float = 0.5):
    """
    하나의 로그 파일 경로를 받아서
    - 파일 이름 기반 파라미터
    - summary 블럭 내용
    - per-step metrics
    다 합쳐서 dict 로 리턴.
    """
    df, summary = load_run_log(path)
    params   = parse_params_from_filename(path)
    metrics  = compute_metrics(df, safe_dist=safe_dist)
    summary_parsed = parse_summary_types(summary)
    
    row = {}
    row.update(params)
    row.update(summary_parsed)
    row.update(metrics)
    row["log_path"] = path
    
    return row


In [7]:
rows = []
for p in all_files:
    row = analyze_log(p, safe_dist=0.5)
    rows.append(row)

summary_df = pd.DataFrame(rows)
print("정리된 행 개수:", len(summary_df))
summary_df.head()


정리된 행 개수: 12


,FFG,SCD,Gap,Bub,Clr,WW,AW,SW,HB,CT,...,Min Dist (m),Unsafe Ratio,FGM Min Dist (m),FGM Unsafe Ratio,FGM Track RMS (m),total_time,collision_flag,min_dist,unsafe_ratio,log_path
0,190.0,35.0,1.5,0.70,0.75,0.4,4.0,0.13,3.0,0.35,...,0.000000e+00,1.0,1.000000e+09,0.0,0.0,19.191206,0,0.0,1.0,./logs/fgm\Opp_bumper_slow_1.csv_FFG190.0_SCD3...
1,180.0,30.0,1.3,0.55,0.60,0.5,5.0,0.10,2.5,0.30,...,0.000000e+00,1.0,1.000000e+09,0.0,0.0,13.612975,0,0.0,1.0,./logs/fgm\Opp_bumper_slow_1.csv_FFG180.0_SCD3...
2,180.0,25.0,1.2,0.50,0.55,0.6,6.0,0.08,2.0,0.25,...,0.000000e+00,1.0,1.000000e+09,0.0,0.0,9.897484,0,0.0,1.0,./logs/fgm\Opp_bumper_slow_1.csv_FFG180.0_SCD2...
3,180.0,25.0,1.3,0.60,0.55,0.5,5.5,0.08,2.0,0.25,...,1.000000e+09,0.0,1.000000e+09,0.0,0.0,0.000000,0,NaN,NaN,./logs/fgm\Opp_bumper_slow_0.5.csv_FFG180.0_SC...
4,180.0,25.0,1.0,0.40,0.45,0.7,8.0,0.05,1.5,0.20,...,0.000000e+00,1.0,1.000000e+09,0.0,0.0,9.858659,0,0.0,1.0,./logs/fgm\Opp_bumper_slow_1.csv_FFG180.0_SCD2...


In [8]:
def get_first_existing(row, candidates, default=np.nan):
    for c in candidates:
        if c in row and not pd.isna(row[c]):
            return row[c]
    return default


In [9]:
def compute_safety_comfort_score(row,
                                 d_safe=0.5,
                                 d_good=0.8,
                                 track_rms_bad=1.0,
                                 w_safety=0.6,
                                 w_progress=0.3,
                                 w_comfort=0.1):
    """
    progress는 아직 모르는 상태라 safety+comfort만 보고 임시 점수 계산.
    (progress는 나중에 duration min/max로 정규화해서 반영)
    """
    # 충돌 / 비정상 종료 체크
    result = str(row.get("result", "")).upper()
    collision_flag = 0
    if "Collision" in row:
        collision_flag = int(row["Collision"])
    elif "collision_flag" in row:
        collision_flag = int(row["collision_flag"])
    
    if collision_flag == 1 or result in ("CRASH", "STUCK", "ABORT"):
        return 0.0
    
    # ----- Safety -----
    fgm_min = get_first_existing(row, ["FGM Min Dist (m)", "fgm_min_dist", "Min Dist (m)", "min_dist"], default=np.nan)
    if math.isnan(fgm_min):
        fgm_min = d_safe
    
    fgm_unsafe = get_first_existing(row, ["FGM Unsafe Ratio", "fgm_unsafe_ratio", "Unsafe Ratio", "unsafe_ratio"], default=0.0)
    if math.isnan(fgm_unsafe):
        fgm_unsafe = 0.0
    
    # 최소 거리 점수
    Sd = (fgm_min - d_safe) / (d_good - d_safe) if d_good > d_safe else 1.0
    Sd = max(0.0, min(1.0, Sd))
    
    # Unsafe Ratio 점수
    Su = 1.0 - max(0.0, min(1.0, fgm_unsafe))
    
    S_safety = 0.7 * Sd + 0.3 * Su
    
    # ----- Comfort (트랙 RMS) -----
    track_rms = get_first_existing(row, ["FGM Track RMS (m)", "fgm_track_rms"], default=0.0)
    if math.isnan(track_rms):
        track_rms = 0.0
    
    S_track = 1.0 - max(0.0, min(1.0, track_rms / track_rms_bad))
    S_comfort = S_track
    
    # progress 제외 상태에서 임시 점수 (safety+comfort 비율만 유지)
    # w_progress는 일단 무시하고, (w_safety + w_comfort)만 사용
    w_sc = w_safety + w_comfort
    Score0 = 0.0
    if w_sc > 0:
        Score0 = (w_safety * S_safety + w_comfort * S_comfort) / w_sc
    return Score0 * 100.0


In [12]:
# 1) safety/comfort 기반 임시 점수 계산
summary_df["Score_tmp"] = summary_df.apply(compute_safety_comfort_score, axis=1)

# 2) duration_raw 칼럼 준비 (Summary > 파일명 순으로)
summary_df["duration_raw"] = summary_df.apply(
    lambda r: get_first_existing(r, ["Duration (s)", "duration_from_name"], default=np.nan),
    axis=1
)

# 3) duration min/max (충돌/ABORT 제외)
valid_mask = (summary_df["Score_tmp"] > 0) & (~summary_df["duration_raw"].isna())
if valid_mask.any():
    T_min = summary_df.loc[valid_mask, "duration_raw"].min()
    T_max = summary_df.loc[valid_mask, "duration_raw"].max()
else:
    T_min, T_max = np.nan, np.nan

def compute_progress(duration, T_min, T_max):
    if math.isnan(duration) or math.isnan(T_min) or math.isnan(T_max) or T_max <= T_min:
        return 0.0
    val = (T_max - duration) / (T_max - T_min)
    return max(0.0, min(1.0, val))

summary_df["S_progress"] = summary_df["duration_raw"].apply(
    lambda t: compute_progress(t, T_min, T_max)
)

def compute_final_score(row,
                        w_safety=0.6,
                        w_progress=0.3,
                        w_comfort=0.1):
    # 충돌 / 비정상 종료면 0점
    result = str(row.get("result", "")).upper()
    collision_flag = 0
    if "Collision" in row:
        collision_flag = int(row["Collision"])
    elif "collision_flag" in row:
        collision_flag = int(row["collision_flag"])
    if collision_flag == 1 or result in ("CRASH", "STUCK", "ABORT"):
        return 0.0
    
    # safety+comfort는 Score_tmp 에 이미 들어 있음 (0~100)
    Score_tmp = row["Score_tmp"]
    # progress는 0~1
    S_progress = row["S_progress"]
    
    # 안전+컴포트 점수 비율 = (1 - w_progress)
    # 최종: Score = (1 - w_progress)*Score_tmp + w_progress*(100 * S_progress)
    Score_final = (1.0 - w_progress) * Score_tmp + w_progress * (100.0 * S_progress)
    
    # 클립
    return max(0.0, min(100.0, Score_final))

summary_df["Score"] = summary_df.apply(compute_final_score, axis=1)

# 점수 높은 순으로 보기
summary_df_sorted = summary_df.sort_values("Score", ascending=False)
summary_df_sorted.head(10)[[
    "scenario", "result",
    "FFG", "SCD", "Gap", "Bub", "Clr", "WW", "AW", "SW", "HB", "CT", "SA", "DBS", "SPD",
    "Score",
    "FGM Min Dist (m)","FGM Unsafe Ratio","FGM Track RMS (m)",
    "Duration (s)","duration_from_name"
]]


,scenario,result,FFG,SCD,Gap,Bub,Clr,WW,AW,SW,...,CT,SA,DBS,SPD,Score,FGM Min Dist (m),FGM Unsafe Ratio,FGM Track RMS (m),Duration (s),duration_from_name
4,Opp_bumper_slow_1.csv_FFG180.0_SCD25.0,GOAL,180.0,25.0,1.0,0.40,0.45,0.7,8.0,0.05,...,0.20,0.6,0.18,5.5,100.000000,1.000000e+09,0.0,0.0,9.86,9.86
5,Opp_bumper_slow_1.5.csv_FFG180.0_SCD25.0,GOAL,180.0,25.0,1.2,0.50,0.55,0.6,6.0,0.08,...,0.25,0.5,0.12,4.0,99.903537,1.000000e+09,0.0,0.0,9.89,9.89
2,Opp_bumper_slow_1.csv_FFG180.0_SCD25.0,GOAL,180.0,25.0,1.2,0.50,0.55,0.6,6.0,0.08,...,0.25,0.5,0.12,4.0,99.871383,1.000000e+09,0.0,0.0,9.90,9.90
10,Opp_bumper_slow_0.5.csv_FFG180.0_SCD25.0,GOAL,180.0,25.0,1.0,0.40,0.45,0.7,8.0,0.05,...,0.20,0.6,0.18,5.5,99.839228,1.000000e+09,0.0,0.0,9.91,9.91
9,Opp_bumper_slow_0.5.csv_FFG180.0_SCD25.0,GOAL,180.0,25.0,1.2,0.50,0.55,0.6,6.0,0.08,...,0.25,0.5,0.12,4.0,99.614148,1.000000e+09,0.0,0.0,9.98,9.98
8,Opp_bumper_slow_1.5.csv_FFG180.0_SCD30.0,GOAL,180.0,30.0,1.3,0.55,0.60,0.5,5.0,0.10,...,0.30,0.4,0.15,3.5,96.270096,1.000000e+09,0.0,0.0,11.02,11.02
11,Opp_bumper_slow_0.5.csv_FFG180.0_SCD30.0,GOAL,180.0,30.0,1.3,0.55,0.60,0.5,5.0,0.10,...,0.30,0.4,0.15,3.5,89.774920,1.000000e+09,0.0,0.0,13.04,13.04
1,Opp_bumper_slow_1.csv_FFG180.0_SCD30.0,GOAL,180.0,30.0,1.3,0.55,0.60,0.5,5.0,0.10,...,0.30,0.4,0.15,3.5,87.942122,1.000000e+09,0.0,0.0,13.61,13.61
0,Opp_bumper_slow_1.csv_FFG190.0_SCD35.0,GOAL,190.0,35.0,1.5,0.70,0.75,0.4,4.0,0.13,...,0.35,0.3,0.22,3.0,70.000000,1.000000e+09,0.0,0.0,19.19,19.19
7,Opp_bumper_slow_0.5.csv_FFG190.0_SCD35.0,GOAL,190.0,35.0,1.5,0.70,0.75,0.4,4.0,0.13,...,0.35,0.3,0.22,3.0,70.000000,1.000000e+09,0.0,0.0,19.19,19.19


In [13]:
param_cols = [    "FFG", "SCD", "Gap", "Bub", "Clr", "WW", "AW", "SW", "HB", "CT", "SA", "DBS", "SPD"]

grouped = (
    summary_df
    .groupby(param_cols)
    .agg(
        mean_score=("Score", "mean"),
        min_score=("Score", "min"),
        max_score=("Score", "max"),
        runs=("Score", "count"),
        crashes=("Collision", "sum") if "Collision" in summary_df.columns else ("collision_flag", "sum")
    )
    .reset_index()
    .sort_values("mean_score", ascending=False)
)

grouped


,FFG,SCD,Gap,Bub,Clr,WW,AW,SW,HB,CT,SA,DBS,SPD,mean_score,min_score,max_score,runs,crashes
0,180.0,25.0,1.0,0.40,0.45,0.7,8.0,0.05,1.5,0.20,0.60,0.18,5.5,99.919614,99.839228,100.000000,2,0
1,180.0,25.0,1.2,0.50,0.55,0.6,6.0,0.08,2.0,0.25,0.50,0.12,4.0,99.796356,99.614148,99.903537,3,0
3,180.0,30.0,1.3,0.55,0.60,0.5,5.0,0.10,2.5,0.30,0.40,0.15,3.5,91.329046,87.942122,96.270096,3,0
5,190.0,35.0,1.5,0.70,0.75,0.4,4.0,0.13,3.0,0.35,0.30,0.22,3.0,70.000000,70.000000,70.000000,2,0
2,180.0,25.0,1.3,0.60,0.55,0.5,5.5,0.08,2.0,0.25,0.75,0.25,4.2,0.000000,0.000000,0.000000,1,0
4,185.0,28.0,1.2,0.50,0.60,0.6,6.0,0.09,3.5,0.40,0.50,0.12,4.0,0.000000,0.000000,0.000000,1,0


In [14]:
import numpy as np
import pandas as pd

# FFG = fgm_fov_angle
# SCD = fgm_speed_check_fov_deg
param_cols = [
    "FFG", "SCD",   # 새로 추가된 두 파라미터
    "Gap", "Bub", "Clr", "WW", "AW", "SW", "HB", "CT", "SA", "DBS", "SPD"
]

# 각 파라미터 튜닝 범위 (대략 이전에 쓰던 값 기준으로 적당히 설정)
param_bounds = {
    "FFG": (140.0, 210.0),   # fov angle [deg]
    "SCD": (10.0,  45.0),    # speed check fov [deg]

    "Gap": (0.8,  2.0),
    "Bub": (0.3,  0.8),
    "Clr": (0.4,  0.8),
    "WW":  (0.2,  1.0),
    "AW":  (2.0, 10.0),
    "SW":  (0.02, 0.2),
    "HB":  (0.0,  4.0),
    "CT":  (0.15, 0.5),
    "SA":  (0.2,  0.8),
    "DBS": (0.05, 0.3),
    "SPD": (2.5,  6.0),
}


In [15]:
rng = np.random.default_rng(42)  # 필요하면 42 말고 다른 숫자로 바꿔도 됨

def propose_new_params_simple(
    grouped: pd.DataFrame,
    param_cols,
    param_bounds,
    n_seeds=3,       # mean_score 상위 몇 세트를 seed로 쓸지
    per_seed=10,     # seed 하나당 몇 개 후보를 생성할지
    noise_scale=0.08 # ±8% 정도 multiplicative noise
):
    """
    grouped: FFG,SCD,Gap~SPD, mean_score 등이 들어있는 DataFrame
    param_cols: 튜닝할 파라미터 컬럼 이름 리스트
    param_bounds: 각 파라미터 별 (min,max) 딕셔너리
    """
    # 1) mean_score 높은 순서대로 상위 n_seeds 뽑기
    top = grouped.sort_values("mean_score", ascending=False).head(n_seeds)
    
    new_rows = []
    for _, seed in top.iterrows():
        base = seed[param_cols].astype(float).values  # seed 파라미터 벡터
        
        for _ in range(per_seed):
            # multiplicative gaussian noise: θ' = θ * (1 + N(0, noise_scale))
            noise = rng.normal(loc=0.0, scale=noise_scale, size=len(param_cols))
            cand = base * (1.0 + noise)
            
            # 각 파라미터 별 bounds 안으로 자르기
            cand_clipped = []
            for val, name in zip(cand, param_cols):
                lo, hi = param_bounds[name]
                cand_clipped.append(float(np.clip(val, lo, hi)))
            
            new_rows.append(dict(zip(param_cols, cand_clipped)))
    
    new_df = pd.DataFrame(new_rows)
    
    # 2) 이미 실험한 파라미터 세트와 완전히 같은 조합은 제거 (중복 방지)
    merged = new_df.merge(grouped[param_cols], on=param_cols, how="left", indicator=True)
    new_unique = merged[merged["_merge"] == "left_only"][param_cols].reset_index(drop=True)
    
    return new_unique


In [16]:
new_candidates = propose_new_params_simple(
    grouped,
    param_cols=param_cols,
    param_bounds=param_bounds,
    n_seeds=3,      # mean_score 상위 3개를 seed로
    per_seed=10,    # seed 하나당 10개씩 → 총 30개 후보
    noise_scale=0.08
)

print("새로 제안된 파라미터 개수:", len(new_candidates))
new_candidates.head()


새로 제안된 파라미터 개수: 30


,FFG,SCD,Gap,Bub,Clr,WW,AW,SW,HB,CT,SA,DBS,SPD
0,184.387926,22.920032,1.060036,0.430098,0.400000,0.627078,8.081818,0.048735,1.497984,0.186351,0.642211,0.191200,5.529054
1,196.232273,25.935019,0.931257,0.411800,0.415480,0.749193,7.968047,0.049261,1.418288,0.219561,0.592583,0.173832,5.345061
2,187.665252,25.730888,1.033019,0.413786,0.527099,0.677241,7.672165,0.046745,1.573918,0.218064,0.594531,0.167902,5.137228
3,189.368536,26.486508,1.043452,0.378704,0.458358,0.706534,8.139961,0.053486,1.526831,0.210863,0.603244,0.184163,5.777767
4,159.016956,24.360658,0.962370,0.379556,0.440095,0.783717,7.445868,0.053873,1.298056,0.194642,0.607812,0.188442,5.812940


In [ ]:
def df_to_params_list_strings(df):
    """
    new_candidates DataFrame(FFG,SCD,Gap~SPD 13개) → bash params_list용 문자열 리스트
    """
    lines = []
    for _, row in df.iterrows():
        vals = [
            row["Gap"],
            row["Bub"],
            row["FFG"],
            row["SCD"],
            row["Clr"],
            row["WW"],
            row["AW"],
            row["SW"],
            row["HB"],
            row["CT"],
            row["SA"],
            row["DBS"],
            row["SPD"],
            
        ]
        s = " ".join(f"{v:.4f}" for v in vals)
        lines.append(f"\"{s}\"")
    return lines

param_strings = df_to_params_list_strings(new_candidates)

print("params_list=(")
for s in param_strings:
    print("    " + s)
print(")")


params_list=(
    "1.0600 0.4301 184.3879 22.9200 0.4000 0.6271 8.0818 0.0487 1.4980 0.1864 0.6422 0.1912 5.5291"
    "0.9313 0.4118 196.2323 25.9350 0.4155 0.7492 7.9680 0.0493 1.4183 0.2196 0.5926 0.1738 5.3451"
    "1.0330 0.4138 187.6653 25.7309 0.5271 0.6772 7.6722 0.0467 1.5739 0.2181 0.5945 0.1679 5.1372"
    "1.0435 0.3787 189.3685 26.4865 0.4584 0.7065 8.1400 0.0535 1.5268 0.2109 0.6032 0.1842 5.7778"
    "0.9624 0.3796 159.0170 24.3607 0.4401 0.7837 7.4459 0.0539 1.2981 0.1946 0.6078 0.1884 5.8129"
    "0.9630 0.4275 191.4242 24.3025 0.4431 0.6286 7.2747 0.0463 1.5597 0.2023 0.6331 0.1738 5.5698"
    "1.0365 0.3788 189.0085 24.3813 0.4369 0.6786 7.2347 0.0519 1.4437 0.2002 0.6231 0.1864 5.7928"
    "0.9936 0.3460 178.5818 24.1534 0.4000 0.6259 7.3618 0.0516 1.3913 0.1939 0.6624 0.1749 5.8245"
    "0.9240 0.3892 166.5559 24.5891 0.4803 0.6033 8.2780 0.0510 1.4287 0.1769 0.6035 0.1724 5.6024"
    "0.9809 0.3672 180.3147 28.2036 0.4565 0.7123 8.8699 0.0533 1.5428 0.2234 0.5429 0